In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# imports & froms

import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.layers import Dense,Dropout,Activation,AveragePooling2D,Input,Conv2D,Flatten,BatchNormalization
from keras.models import Model
from keras.utils import to_categorical
import matplotlib.pyplot as plt


In [0]:
# data load

fileLocations='drive/My Drive/data_sets/bengali_ai/'
tY=np.load(fileLocations+'dY.npy')
tX=np.load(fileLocations+'dX.npy')

In [0]:
# data process

total_tX=tX.shape[0]
total_vX=vX.shape[0]
classRoots=168
classVowels=11
classConsonants=7
height=64
width=80

tX=tX.reshape(total_tX,height,width,1)
vX=vX.reshape(total_vX,height,width,1)
tX=tX/255
vX=vX/255

trY=np.empty([total_tX,1],dtype=np.int32)
vrY=np.empty([total_vX,1],dtype=np.int32)
tvY=np.empty([total_tX,1],dtype=np.int32)
vvY=np.empty([total_vX,1],dtype=np.int32)
tcY=np.empty([total_tX,1],dtype=np.int32)
vcY=np.empty([total_vX,1],dtype=np.int32)

for it in range(total_tX):  
    trY[it,:]=tY[it][0]
    tvY[it,:]=tY[it][1]
    tcY[it,:]=tY[it][2]
for it in range(total_vX):  
    vrY[it,:]=vY[it][0]
    vvY[it,:]=vY[it][1]
    vcY[it,:]=vY[it][2]
  
trY=to_categorical(trY)
tvY=to_categorical(tvY)
tcY=to_categorical(tcY)
vrY=to_categorical(vrY)
vvY=to_categorical(vvY)
vcY=to_categorical(vcY)

In [0]:
# models

def modelRoot(inputX,classes):
    X=Conv2D(8,(3,3),strides=(1,1),activation='relu',name='rootL1')(inputX)
    X=BatchNormalization(axis=-1,name='rootL2')(X)
    X=AveragePooling2D((2,2),name='rootL3')(X)
    
    X=Conv2D(16,(3,3),strides=(1,1),activation='relu',name='rootL5')(X)
    X=BatchNormalization(axis=-1,name='rootL6')(X)
    X=AveragePooling2D((2,2),name='rootL7')(X)
 
    X=Flatten()(X)
    X=Dense(128,activation='relu',name='rootL13')(X)
    X=Dense(64,activation='relu',name='rootL14')(X)
   
    X=Dense(classes,activation='softmax',name='rootOutput')(X)
    return X
    
def modelVowel(inputX,classes):
    X=Conv2D(8,(3,3),strides=(1,1),activation='relu',name='vowelL1')(inputX)
    X=BatchNormalization(axis=-1,name='vowelL2')(X)
    X=AveragePooling2D((2,2),name='vowelL3')(X)

    
    X=Conv2D(16,(3,3),strides=(1,1),activation='relu',name='vowelL5')(X)
    X=BatchNormalization(axis=-1,name='vowelL6')(X)
    X=AveragePooling2D((2,2),name='vowelL7')(X)
    
    X=Flatten()(X)
    X=Dense(128,activation='relu',name='vowelL9')(X)
    X=Dense(64,activation='relu',name='vowelL10')(X)
    
    X=Dense(classes,activation='softmax',name='vowelOutput')(X)
    return X
    
    
def modelConsonant(inputX,classes):
    X=Conv2D(8,(3,3),strides=(1,1),activation='relu',name='consonantL1')(inputX)
    X=BatchNormalization(axis=-1,name='consonantL2')(X)
    X=AveragePooling2D((2,2),name='consonantL3')(X)
    
    X=Conv2D(16,(3,3),strides=(1,1),activation='relu',name='consonantL5')(X)
    X=BatchNormalization(axis=-1,name='consonantL6')(X)
    X=AveragePooling2D((2,2),name='consonantL7')(X)
    
    X=Flatten()(X)
    X=Dense(128,activation='relu',name='consonantL9')(X)
    X=Dense(64,activation='relu',name='consonantL10')(X)
    
    X=Dense(classes,activation='softmax',name='consonantOutput')(X)
    return X


In [0]:
# merge model & compile

inputShape= Input((height,width,1))

mRoot=modelRoot(inputShape,classRoots)
mVowel=modelVowel(inputShape,classVowels)
mConsonant=modelConsonant(inputShape,classConsonants)

model=Model(inputs=inputShape,outputs=[mRoot,mVowel,mConsonant],name='banglaNet')

losses={
    'rootOutput':'categorical_crossentropy',
    'vowelOutput':'categorical_crossentropy',
    'consonantOutput':'categorical_crossentropy'
}

weights={
    'rootOutput':1.0,
    'vowelOutput':1.0,
    'consonantOutput':1.0
}

outputs={'rootOutput':trY,
         'vowelOutput':tvY,
         'consonantOutput':tcY
}

vY={'rootOutput':vrY,
    'vowelOutput':vvY,
    'consonantOutput':vcY
}

model.compile(optimizer='adam',loss=losses,loss_weights=weights,metrics=['accuracy'])

In [0]:
#run model
Epoches=4
history=model.fit(tX,outputs,validation_data=(vX,vY),epochs=Epoches,verbose=1)

In [0]:
#evaluate

result=model.evaluate(x=vX, y=vY, batch_size=64, verbose=1)

In [0]:
#save model

model_json = model.to_json()
with open(fileLocations+'modelAug.json', "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(fileLocations+'modelWeight.h5')

In [0]:
#load model

from keras.models import model_from_json
json_file = open(fileLocations+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights(fileLocations+'modelWeight.h5')